# FreelyMovingEphys modular DeepLabCut data intake
## modular_dlc_intake.ipynb

DeepLabCut points and associated timestamps and videos are read in for any or no inputs of the possible camera types: world, eye, and topdown.

This script, modular_dlc_intake.ipynb, is an alternative means of accessing the underlying functions to the more high-throughput terminal interface in dlc_intake.py

Last modified July 10, 2020

In [1]:
# module imports
from nb_wrapper import topdown_intake, eye_intake
from util.save_data import savetrial

## global user inputs

In [2]:
# source for .avi videos, DLC .h5 files, and .csv timestamp files
global_data_path = '/home/dylan/data/Niell/FreelyMovingEphys/tests/070920/test1/'
# path into which outputs will be saved
global_save_path = '/home/dylan/data/Niell/FreelyMovingEphys/outputs/tests/070920/test1/run1/'
# DLC likelihood threshold
lik_thresh = 0.99
# max number of pixels for radius of pupil
pxl_thresh = 50
# maximum ratio of ellipse shortaxis to longaxis
ell_thresh = 0.90
# value with which to correct y-coordinates in topdown view
coord_cor = 0
# number of labeled topdown DeepLabCut points (including points on cricket, if there is one)
topdown_pt_num = 10
# if there is a cricket in the tank as the last two DeepLabCut points (True/False)
cricket = True
# number of labeled eye camera points (including tear duct and back of eye, if those exist)
eye_pt_num = 8
# if there are eye points labeled for the tear duct and back of the eye (True/False)
tear = False
# use BonsaiTS timestamps (True) or FlirTS timestamps (False)
bonsaitime = True

## topdown views
The topdown_intake function will read in a single topdown camera view's DLC data and videos, threshold the values based on likelihood, and save the data out.
The function should be run on each camera seperately, and each output of topdown_intake should be saved with savex individually so that there is one .nc file for every topdown view (i.e. TOP1, TOP2, and TOP3).
Do not include the file extension in the name string. The extensions '.avi', '_BonsaiTS.csv', and '.h5' will be added as needed.

In [3]:
# the name of the file of topdown view, which should be shared between time, video, and DLC inputs
# e.g. '070920_test1_EphysSaline_TOP1'
topdown1 = '070920_test1_EphysSaline_TOP1'

xtop1, xtopTS1 = topdown_intake(global_data_path, topdown1, global_save_path, lik_thresh, coord_cor, topdown_pt_num, cricket, bonsaitime)

missing DLC file


AttributeError: 'NoneType' object has no attribute 'to_netcdf'

Next, the outputs of the topdown_intake function can be saved. The videos will have already been saved within the function to the approriate directory. The returned DLC xarray object that is returned first out of the function is type None if no real DLC data was found. The line to save that object (below) will raise an error if it gets an obeject of type None. The same is true of the time object, which cannot be saved when it is None.

In [ ]:
# make one Dataset for each viewpoint
savetrial(xtop1, global_save_path, topdown1, 'TOP1')
savetrial(xtopTS1, global_save_path, topdown1, 'TOP1time')

## eyecam views
The eye_intake function will read in a single eye's DLC data and videos, threshold by likeihood, get out ellipse paramaters including phi and theta, and save a video of the eye with the ellipse plotted on it.

In [ ]:
# the name of the file of eye view, which should be shared between time, video, and DLC inputs
eyeL = '010120_mouse1_experiment1_LEye'

xeyeL = eye_intake(global_data_path, eyeL, global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)

## worldcam views